# Evolver Loop 2 Analysis

## Critical Discovery: Pre-optimized Baselines Available!

The evaluator identified pre-optimized CSVs that score much better than our current 88.14:
- `/home/nonroot/snapshots/santa-2025/21116303805/code/preoptimized/santa-2025.csv` → Score: ~70.68
- `/home/nonroot/snapshots/santa-2025/21116303805/code/preoptimized/telegram/71.97.csv` → Score: ~71.97
- `/home/nonroot/snapshots/santa-2025/21116303805/code/preoptimized/telegram/72.49.csv` → Score: ~72.49

Let's verify these scores and understand the gap to target (68.92).

In [1]:
import pandas as pd
import numpy as np
from decimal import Decimal, getcontext
from shapely import affinity
from shapely.geometry import Polygon
from shapely.ops import unary_union
from shapely.strtree import STRtree

getcontext().prec = 30
scale_factor = Decimal('1e15')

print("Libraries loaded")

Libraries loaded


In [2]:
class ChristmasTree:
    """Represents a single, rotatable Christmas tree of a fixed size."""

    def __init__(self, center_x='0', center_y='0', angle='0'):
        self.center_x = Decimal(str(center_x))
        self.center_y = Decimal(str(center_y))
        self.angle = Decimal(str(angle))

        trunk_w = Decimal('0.15')
        trunk_h = Decimal('0.2')
        base_w = Decimal('0.7')
        mid_w = Decimal('0.4')
        top_w = Decimal('0.25')
        tip_y = Decimal('0.8')
        tier_1_y = Decimal('0.5')
        tier_2_y = Decimal('0.25')
        base_y = Decimal('0.0')
        trunk_bottom_y = -trunk_h

        initial_polygon = Polygon(
            [
                (Decimal('0.0') * scale_factor, tip_y * scale_factor),
                (top_w / Decimal('2') * scale_factor, tier_1_y * scale_factor),
                (top_w / Decimal('4') * scale_factor, tier_1_y * scale_factor),
                (mid_w / Decimal('2') * scale_factor, tier_2_y * scale_factor),
                (mid_w / Decimal('4') * scale_factor, tier_2_y * scale_factor),
                (base_w / Decimal('2') * scale_factor, base_y * scale_factor),
                (trunk_w / Decimal('2') * scale_factor, base_y * scale_factor),
                (trunk_w / Decimal('2') * scale_factor, trunk_bottom_y * scale_factor),
                (-(trunk_w / Decimal('2')) * scale_factor, trunk_bottom_y * scale_factor),
                (-(trunk_w / Decimal('2')) * scale_factor, base_y * scale_factor),
                (-(base_w / Decimal('2')) * scale_factor, base_y * scale_factor),
                (-(mid_w / Decimal('4')) * scale_factor, tier_2_y * scale_factor),
                (-(mid_w / Decimal('2')) * scale_factor, tier_2_y * scale_factor),
                (-(top_w / Decimal('4')) * scale_factor, tier_1_y * scale_factor),
                (-(top_w / Decimal('2')) * scale_factor, tier_1_y * scale_factor),
            ]
        )
        rotated = affinity.rotate(initial_polygon, float(self.angle), origin=(0, 0))
        self.polygon = affinity.translate(rotated,
                                          xoff=float(self.center_x * scale_factor),
                                          yoff=float(self.center_y * scale_factor))

print("ChristmasTree class defined")

ChristmasTree class defined


In [3]:
def get_tree_list_side_length(tree_list):
    all_polygons = [t.polygon for t in tree_list]
    bounds = unary_union(all_polygons).bounds
    return Decimal(max(bounds[2] - bounds[0], bounds[3] - bounds[1])) / scale_factor

def get_score(trees, n):
    if not trees:
        return 0.0
    side = get_tree_list_side_length(trees)
    return float(side ** 2 / Decimal(n))

def has_overlap(trees):
    if len(trees) <= 1:
        return False
    polygons = [t.polygon for t in trees]
    tree_index = STRtree(polygons)
    for i, poly in enumerate(polygons):
        indices = tree_index.query(poly)
        for idx in indices:
            if idx == i:
                continue
            if poly.intersects(polygons[idx]) and not poly.touches(polygons[idx]):
                return True
    return False

def load_configuration_from_df(n, df):
    group_data = df[df["id"].str.startswith(f"{n:03d}_")]
    trees = []
    for _, row in group_data.iterrows():
        x = str(row["x"])[1:] if str(row["x"]).startswith('s') else str(row["x"])
        y = str(row["y"])[1:] if str(row["y"]).startswith('s') else str(row["y"])
        deg = str(row["deg"])[1:] if str(row["deg"]).startswith('s') else str(row["deg"])
        if x and y and deg:
            trees.append(ChristmasTree(x, y, deg))
    return trees

print("Helper functions defined")

Helper functions defined


In [4]:
# Score the pre-optimized CSVs
csv_files = [
    ('/home/nonroot/snapshots/santa-2025/21116303805/code/preoptimized/santa-2025.csv', 'santa-2025.csv'),
    ('/home/nonroot/snapshots/santa-2025/21116303805/code/preoptimized/telegram/71.97.csv', '71.97.csv'),
    ('/home/nonroot/snapshots/santa-2025/21116303805/code/preoptimized/telegram/72.49.csv', '72.49.csv'),
    ('/home/nonroot/snapshots/santa-2025/21116303805/code/preoptimized/ensemble.csv', 'ensemble.csv'),
]

for csv_path, name in csv_files:
    try:
        df = pd.read_csv(csv_path)
        total_score = 0
        overlap_count = 0
        
        for n in range(1, 201):
            trees = load_configuration_from_df(n, df)
            if trees:
                score = get_score(trees, n)
                total_score += score
                if has_overlap(trees):
                    overlap_count += 1
        
        print(f"{name}: Score = {total_score:.6f}, Overlaps = {overlap_count}")
    except Exception as e:
        print(f"{name}: Error - {e}")

santa-2025.csv: Score = 70.676102, Overlaps = 12


71.97.csv: Score = 71.972027, Overlaps = 0


72.49.csv: Score = 72.495739, Overlaps = 0


ensemble.csv: Score = 70.676102, Overlaps = 13


In [5]:
# Compare with our current best
print("\n=== SCORE COMPARISON ===")
print(f"Target:                  68.922808")
print(f"Pre-optimized baseline:  ~70.68 (gap: 1.76 points)")
print(f"Our grid+bbox3:          88.14 (gap: 19.22 points)")
print(f"Our greedy baseline:     164.09 (gap: 95.17 points)")
print("\n==> Using pre-optimized baseline saves 17.46 points immediately!")


=== SCORE COMPARISON ===
Target:                  68.922808
Pre-optimized baseline:  ~70.68 (gap: 1.76 points)
Our grid+bbox3:          88.14 (gap: 19.22 points)
Our greedy baseline:     164.09 (gap: 95.17 points)

==> Using pre-optimized baseline saves 17.46 points immediately!


In [6]:
# Check what's in the chistyakov folder
import os
for root, dirs, files in os.walk('/home/nonroot/snapshots/santa-2025/21116303805/code/preoptimized/chistyakov/'):
    for f in files:
        print(os.path.join(root, f))

/home/nonroot/snapshots/santa-2025/21116303805/code/preoptimized/chistyakov/submission_best.csv
/home/nonroot/snapshots/santa-2025/21116303805/code/preoptimized/chistyakov/santa2025-packed-version-of-current-best-public.zip


In [7]:
# Check chistyakov submission
chistyakov_path = '/home/nonroot/snapshots/santa-2025/21116303805/code/preoptimized/chistyakov/submission_best.csv'
df = pd.read_csv(chistyakov_path)
total_score = 0
overlap_count = 0

for n in range(1, 201):
    trees = load_configuration_from_df(n, df)
    if trees:
        score = get_score(trees, n)
        total_score += score
        if has_overlap(trees):
            overlap_count += 1

print(f"chistyakov/submission_best.csv: Score = {total_score:.6f}, Overlaps = {overlap_count}")

chistyakov/submission_best.csv: Score = 70.926150, Overlaps = 0
